In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
# Define the input image size (224x224 for ResNet50)
input_size = (224, 224)

# Provide the path to your dataset
train_path = 'F:/Dataset/train'
test_path = 'F:/Dataset/val'

# Create the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=input_size,
    batch_size=32,
    label_mode='categorical',
    shuffle=True
)

# Create the testing dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=input_size,
    batch_size=32,
    label_mode='categorical',
    shuffle=False  # Set to False for testing/validation dataset
)

# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create the Sequential model
model = Sequential()

# Add the pre-trained ResNet50 model
model.add(base_model)

# Add custom top layers for the specific classification task
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(38, activation='softmax'))  # 38 classes for your specific task

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_dataset,
    epochs=10,  # Adjust the number of epochs based on your specific requirements
    validation_data=test_dataset
)

Found 43444 files belonging to 38 classes.
Found 10861 files belonging to 38 classes.


Epoch 1/10
1175/1358 [========================>.....] - ETA: 7:07 - loss: 0.3214 - accuracy: 0.9006

In [3]:
model.save('NewModel.h5')